# Dynamic Programming (動的計画法)

競プロにて頻繁に登場するアルゴリズム  
簡単な問題を通じて実装をしていく

Reference
- 競技プログラミングの鉄則
    - 書籍。図が多くて分かりやすい。

動的計画法は一言で表すと、より小さい問題の結果を利用して問題を解く方法などとよく言われる。  
が、そういわれても(自分は)ピンとこないので実際に具体例を見る方が多分手っ取り早い。

## 例題1 動的計画法の基本
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_p

---
以下解法とか、考え方とか

- dpの作り方  
dp[i]を部屋1から部屋iまでの最短時間とする  
    - 今回のような簡単な例では自然に思いつきやすいが、ここのdpをどう作るかが難しく色々なパターンがある  
- dp[i]の計算の仕方  
dp[1]は移動する必要がないから0(部屋は1-index)  
dp[2]は1から$A_2$時間かけて移動するしかないのでdp[2]=$A_2$
dp[3]は部屋1から$A_2 + A_3$かけて移動するか、$B_3$かけて移動するかの2通り  
もっと言うと1個前の部屋からの移動時間dp[2] + $A_3$か2個前部屋からの移動時間dp[1] + $B_3$と表せる  
一般化すると部屋iは1個前の部屋からdp[i-1] + $A_i$かけて移動するか、dp[i-2] + $B_i$かけて移動するかの2通りとなる  
    -  dpは最短時間を計算してあるはずなので考えるのはこの2通りだけで、それ以前は考えなくて大丈夫  
従って
$$
dp[1] = 0  \\
dp[2] = A_2 \\
dp[i] = \min (dp[i-1] + A_i, dp[i-2] + B_i) \\
$$
と表せ、これを手前から(0からN方向へ)計算していけば計算量$O(N)$で求まる


In [13]:
# 回答例

# 入力
N = 5
a = [2,4,1,3]
b = [5,3,7]

# 回答
dp = [] # numpy使ってdp = np.zeros(shape=(N,))とかしてdp[i] = の形にしても良い。そっちの方が分かりやすい
# 以下0-index (分かりやすくしたいなら1-indexになるよう番兵とか入れて調整して)
dp.append(0) # dp[1] = 0
dp.append(a[0]) # dp[2] = A_2
for i in range(2,N):
    dp.append(min(dp[i-1]+a[i-1],dp[i-2]+b[i-2]))

print(dp[-1])

8


In [14]:
# 添え字がバチクソ分かりにくいからnumpyの配列を使うバージョンも
# ついでに問題に合わせて1-indexにしちゃう
import numpy as np
dp = np.zeros(shape=(N+1,),dtype=np.int64)
a = [0,0] + a
b = [0,0,0] + b

dp[0] = 1e9
dp[1] = 0
dp[2] = a[2]
for i in range(3,N+1):
    dp[i] = min(dp[i-1]+a[i],dp[i-2]+b[i])
print(dp[-1])


8


## 例題2 動的計画法の復元
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_q

---
先の問題では手前から順にdp[i]を更新していった  
動的計画法の経路の復元では逆にゴール側からdp[i]を計算するときにどっちのルートからやってきたかを辿ることで復元を行う  
具体的には  
dp[N]はd[N-1] + $A_N$ or d[N-2] + $B_N$を計算することで合う方がdp[N]の1つ手前の状態であることが分かる  
d[N-2] + $B_N$ = dp[N]なら1つ前はN-2なので今度はdp[N-2]はdp[N-2-1] + $A_{N-2}$ or dp[N-2-2] + $B_{N-2}$を計算することで
どっちのルートからやってきたか分かる  
これを続けてスタートまで求めることで経路の復元ができる

In [15]:
# 回答例
# 入力
N = 5
a = [2,4,1,3]
b = [5,3,7]

import numpy as np
# 動的計画法の計算
dp = np.zeros(shape=(N+1,),dtype=np.int64)
a = [0,0] + a
b = [0,0,0] + b

dp[1] = 0
dp[2] = a[2]
for i in range(3,N+1):
    dp[i] = min(dp[i-1]+a[i],dp[i-2]+b[i])

# ここから経路の計算
route = []
pos = N
while pos!= 1:
    route.append(pos)
    if dp[pos] == dp[pos-1] + a[pos]:
        pos = pos-1
    elif dp[pos] == dp[pos-2] + b[pos]:
        pos = pos-2

route.append(pos)
print(*route[::-1])

1 2 4 5


## 例題3 2次元のDP(1): 部分和問題  
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_r  

---
単純な解法だとN毎のカードを選ぶか、選ばないかで全探索することになるが時間計算量が$O(2^N)$になる  
そこでDPを考える(どういう風にDPを設計、考えるか、が大事)  
今回の問題では  
dp[i][j]: カードをi枚目まで使ったときに整数jを作れるか  
とdpを二次元配列で作ってやると上手くいく  
i=0(カードを一枚も使わない)のときは当然j=0のみできるのでdp[0][0]=1  
i>0ではi-1番目でできる整数に対して、a[i]のカードを使うか、使わないかの2パターンの整数の作り方が発生しそれ以外の整数は作り得ない  

In [16]:
# 回答例
# 入力
N=3
S=7
A=[2,2,3]

import numpy as np
A = [0] + A # 1-index化
dp = np.zeros(shape=(N+3,S+3),dtype=np.int64) # 番兵：あって保険になっても損はしないのでn+1ではなくn+3まで用意してある


dp[0][0] = 1 # カードを0枚使ってできるのは0のみ
for i in range(1,N+1): # 手前から順にA[i]のカードを使うかどうか
    for j in range(0,S+1): # i枚目のカードを使ったとき
        if dp[i-1,j]==1: # i-1枚でjが作れたら
            dp[i,j] = 1 # i枚目でA[i]を使わないパターンか
            if j+A[i] > S+1:
                continue # (Sを越えたらもうその先は必要ない (dp配列を越えてエラーもでるし))
            dp[i,j+A[i]] = 1 # i枚目でA[i]を使うパターンができる

if dp[N,S]==1:
    print("Yes")
else:
    print("No")

Yes


## 例題4　2次元のDP(2): ナップザック問題
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_s

---
愚直な解法では品物の選び方$2^N$通りを全探索することがあるが、Nが大きいと計算量が当然大きくなる。  
そこでDPを考える。  
品物を1個、2個、...、N個まで選ぶことを考えてその中で重さの合計がjとなるような価値の最大値をdp[i][j]とする。  

まずi=0のとき、何も選べないのでdp[0][0]=0、j>0は選べないので-1としておく。  
i=1のとき、i=0の（全ての）状態から品物1を選ぶか選ばないかを考えることでdp[1][j] (j=0,1,...,W)を求めることができる。  
i=2のときも同様にi=1の状態から品物2を選ぶか選ばないかを考えることでdp[2][j] (j=0,1,...,W)を求めるこができる。  
以下同様にi=nのときにi=n-1の状態から品物nを選ぶか選ばないかを考えることでdp[n][j] (j=0,1,...,W)を求めることができる。  
これによりO(NW)で解ける。  


In [17]:
# 回答例
# 入力
N = 4
W = 7
wv = [(3,13),(3,17),(5,29),(1,10)]

dp = [[-1e6]*(W+1) for _ in range(N+1)]
dp[0][0] = 0
for i in range(N):
    for j in range(W):
        next_weight = wv[i][0]
        next_value = wv[i][1]
        # それぞれの前の状態から選ぶ場合と選ばない場合についてdpを計算
        if dp[i][j] >= 0: # 状態が存在して
            # 選ばない場合
            dp[i+1][j] = max(dp[i+1][j], dp[i][j])

            # 選ぶ場合は重さがWを超えない場合に限る
            if j+next_weight <= W:
                dp[i+1][j+next_weight] = max(dp[i+1][j+next_weight], dp[i][j] + next_value)

ans = max([max(dp[i]) for i in range(N+1)])
print(ans)


40


## 例題5　2次元のDP(3): 最長共通部分列問題
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_t

---
縦軸に文字列1を、横軸に文字列2をびゃーっと並べると見通しが良くなる。  
共通文字列の探索はマス目を左上から右下に向かっていくときに、同じ文字があった場合に斜めに+1となる。縦横で進んでいくときに最も斜め移動が多くなるように進めば求まる。  
dp[i][j]: 文字列1のi文字目までと文字列2のj文字目までの最長共通部分列の長さ  
とする。  
dp[i][0] = dp[0][j] = 0 (i=0,1,...,N, j=0,1,...,M)と最初に初期化することができ以降は
dp[i][j] = dp[i-1][j-1] + 1 (if (s[i] = t[j]))  
と更新することでO(NM)で解ける。

In [9]:
# 回答例
# 入力
S = "tokyo"
T = "kyoto"

dp = [[0]*(len(T)+1) for _ in range(len(S)+1)] # 全て0で初期化で問題なし

for i in range(len(S)+1):
    for j in range(len(T)+1):
        if i == 0 or j == 0:
            continue
        if S[i-1] == T[j-1]: # 文字列の添え字とずれているので注意
            tmp = max(dp[i-1][j], dp[i][j-1])
            dp[i][j] = max(dp[i-1][j-1]+1, tmp)
        else:
            dp[i][j] = max(dp[i-1][j], dp[i][j-1])

print(np.array(dp))
print(dp[len(S)][len(T)]) # dp[-1][-1]でも良い

[[0 0 0 0 0 0]
 [0 0 0 0 1 1]
 [0 0 0 1 1 2]
 [0 1 1 1 1 2]
 [0 1 2 2 2 2]
 [0 1 2 3 3 3]]
3


In [3]:
import numpy as np